In [34]:
from threading import Thread
from queue import Queue, Empty
from subprocess import Popen, PIPE
from time import sleep
import numpy as np

class NonBlockingStreamReader:

    def __init__(self, stream):
        '''
        stream: the stream to read from.
                Usually a process' stdout or stderr.
        '''

        self._s = stream
        self._q = Queue()

        def _populateQueue(stream, queue):
            '''
            Collect lines from 'stream' and put them in 'quque'.
            '''

            while True:
                line = stream.readline()
                if line:
                    queue.put(line)
                else:
                    return
                    #raise UnexpectedEndOfStream

        self._t = Thread(target = _populateQueue,
                args = (self._s, self._q))
        self._t.daemon = True
        self._t.start() #start collecting lines from the stream

    def readline(self, timeout = None):
        try:
            return self._q.get(block = timeout is not None,
                    timeout = timeout)
        except Empty:
            return None

class UnexpectedEndOfStream(Exception): pass

def output_from_command(command=None):
    if command:
        command = command.encode()
        p.stdin.write(command)
        p.stdin.flush()
    outputList = []
    while True:
        output = nbsr.readline(0.1) # 0.1 secs to let the shell output the result
        if not output:
            outputList.append('No more data\n\n')
            return outputList
        else:
            outputList.append(output)
            
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        elif isinstance(value, list):
            for i in value:
                print('\t' * (indent+1) + str(i))
        else:
            print('\t' * (indent+1) + str(value))

def findID(serapiString):
    start = serapiString.find("Added")
    end = serapiString.find("((")
    thisID = serapiString[start + 6:end]
    return thisID
    
    
def doAdd(coqString, resultDict, debugList = []):
    
    if 10 in debugList:
        debugList = [0,1,2,3]
        
    commandExtended = '(Add () "%s")' % coqString
    
    if 0 in debugList:
        print("Add command: ")
        print(commandExtended)
        print()
        
    addResult = output_from_command(command=commandExtended)[-3].decode('ASCII')
    
    if 1 in debugList:
        print("Add command result: ")
        print(addResult)
        print()
        
    thisID = findID(addResult)
    
    '''
    start = result.find("Added")
    end = result.find("((")
    thisID = result[start + 6:end]
    '''
    
    execCommand = '(Exec %s)' % thisID
    execResult = output_from_command(execCommand)
    
    if 2 in debugList:
        print("Exec result: ")
        print(execResult)
        print()
    
    
    if sum([1 if "Error" in i.decode('ASCII') else 0 for i in execResult if type(i) == bytes]) > 0:
        print("Error...")
        cancelCommand = '(Cancel (%s))' % thisID
        cancelResult = output_from_command(command=cancelCommand)
        print("Cancel result: ")
        print(cancelResult)
        return resultDict
    
    goalCommand = '(Query ((pp ((pp_format PpStr)))) Goals)'
    goalResult = output_from_command(goalCommand)
    
    if 3 in debugList:
        print("Goal Query result: ")
        print(goalResult)
        print()
    
    if len(goalResult) == 1:
        result =  [(['none'],None)]
    else:
        result = goalResult[1].decode('ASCII').replace('\\n','\n')
    
    if '"' not in result:
        result = [(['none'],None)]
    else:
        start = result.find('"')
        result = result[start + 1:]
        end = result.find('"')
        result = result[:end]

        goalList = result.strip().split("\n\n")

        result = [i.split('\n============================\n') for i in goalList]
        result = [(i[0].strip().split('\n'),i[1].replace('\n','')) for i in result]
        result = [([j.strip() for j in i[0]], " ".join(i[1].split())) for i in result]
    
    if coqString in resultDict.keys():
        resultDict[coqString + "     duplicate: " + str(np.random.randint(0,1000))] = result
    else:
        resultDict[coqString] = result
    return resultDict
    
def doCommand(command, resultDict={}):
    if command in resultDict.keys():
        resultDict[command + "     duplicate: " + str(np.random.randint(0,1000))] = output_from_command(command=command)
    else:
        resultDict[command] = output_from_command(command=command)
    return resultDict

try:
    p.terminate()
except:
    pass

p = Popen(["/home/ubuntu/.opam/4.06.1/bin/sertop"], stdin=PIPE, stdout=PIPE, shell=True)
nbsr = NonBlockingStreamReader(p.stdout)

load = output_from_command(None) # omits Coq initialization data from results
resultDict = {}

In [ ]:
coqString = "From Coq Require Import Arith."

In [ ]:
coqString = "Lemma example : forall x y, x <= 10 -> 10 <= y -> x <= y. Proof."

In [ ]:
coqString = "apply le_trans with (m := 10)."

In [ ]:
coqString = "apply le_trans."

In [ ]:
coqString = "Theorem test_prop : forall A B C:Prop, (A -> B -> C) -> (A -> B) -> A -> C.  Proof."

In [ ]:
coqString = "assumption. assumption. Qed."

In [ ]:
coqString = "intros."

In [ ]:
coqString = "Lemma addn0 n : n + 0 = n. Proof. induction n."

In [ ]:
coqString = "Variables A B C: Prop."

In [ ]:
coqString = "Goal (A -> B -> C) -> (A -> B) -> A -> C."

In [35]:
coqString = 'Variables V0 V1 V2: Prop.'

In [39]:
coqString = "Hypothesis A2: V2."

In [41]:
coqString = "Hypothesis I0: V2 -> V1."

In [37]:
coqString = "Goal V1."

In [43]:
coqString = "apply I0."

In [45]:
coqString = "apply A2."

In [46]:
resultDict = doAdd(coqString, resultDict, debugList = [10])
print("Goal dictionary: ")
pretty(resultDict)

Add command: 
(Add () "apply A2.")

Add command result: 
(Answer 15(Added 9((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 9))NewTip))


Exec result: 
[b'(Answer 16 Ack)\n', b'(Feedback((doc_id 0)(span_id 9)(route 0)(contents(ProcessingIn master))))\n', b'(Feedback((doc_id 0)(span_id 8)(route 0)(contents Processed)))\n', b'(Feedback((doc_id 0)(span_id 9)(route 0)(contents Processed)))\n', b'(Answer 16 Completed)\n', 'No more data\n\n']

Goal Query result: 
[b'(Answer 17 Ack)\n', b'(Answer 17(ObjList((CoqString""))))\n', b'(Answer 17 Completed)\n', 'No more data\n\n']



IndexError: list index out of range

In [10]:
resultDict = doCommand(command, resultDict={})
print("Goal dictionary: ")
pretty(resultDict)

Goal dictionary: 
(Cancel (0))
	b'(Answer 4 Ack)\n'
	b'(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))\n'
	b'(Answer 4(Canceled()))\n'
	b'(Answer 4 Completed)\n'
	No more data




In [ ]:
command = '(Add () "Theorem my_first_proof : (forall A : Prop, A -> A).")'

In [ ]:
command = '(Add () "Lemma addn0 n : n + 0 = n. Proof. induction n.")'

In [ ]:
command = '(Add () "Theorem test_prop : forall A B C:Prop, (A -> B -> C) -> (A -> B) -> A -> C.")'

In [ ]:
command = '(Query ((sid 3)) Goals)'

In [ ]:
command = '(Query ( (pp ((pp_format PpStr))) (sid 2) ) Ast)'

In [9]:
command = '(Cancel (0))'

In [ ]:
command = '(Exec 7)'

In [ ]:
command = '(Query () Goals)'

In [ ]:
command = '(Query ((pp ((pp_format PpStr)))) Goals)'

In [ ]:
command = '(Add () "intros.")'

In [ ]:
type(b'hello')

In [ ]:
#resultDict = doCommand(command, resultDict)
#pretty(resultDict)